In [57]:
# import ipinfo
# access_token = '7dbb53d0419093'
# handler = ipinfo.getHandler(access_token)
# ip_address = '95.100.97.48'
# ip_address = '104.24.123.216'
# details = handler.getDetails(ip_address)
# details.city

In [58]:

dic = {
"Washington" : [
14166, 
15542, 
52915, 
54445, 
21003, 
12863, 
32616, 
53565, 
35776, 
55009, 
1001544, 
52574, 
1119, 
10331, 
21802, 
1001197, 
281, 
53097, 
53409, 
1002030, 
22675, 
1001199, 
51304, 
210231101, 
1002640, 
35498, 
4851, 
1004377, 
50142, 
1003891, 
21000, 
1001101, 
13520, 
6264, 
51694, 
6934, 
1000004, 
1000006, 
6947, 
54267, 
1000319, 
55298, 
54233, 
6407, 
1004250, 
1001507, 
34732, 
12681, 
32604, 
50468, 
12622, 
33798, 
55443, 
19911, 
10397,  
14242,  
54931,  
52883,  
12652,  
52883,  
12652,  
12611,  
53097,  
29287,  
25898,  
1058,
1004482,
10486,  
10743, 
16790, 
],
"Arizona" : [  
10905,  
1002766, 
28539, 
28592, 
53929, 
86, 
52326, 
51636, 
50787, 
1002591, 
31116, 
50936, 
1002928, 
6101, 
6408, 
1001552, 
1003541, 
1003531, 
1003533, 
1003535, 
1003537, 
1003538, 
1002799, 
1000732, 
6408, 
1001552, 
23000, 
1002539, 
1000296, 
33076, 
10456, 
12100, 
1002735, 
1002737, 
1002738, 
1004411, 
1001550, 
34770, 
1004540, 
30874, 
10394, 
55688, 
27798, 
19270, 
53929 
], 
"California" : [ 
#SOUTH CAL, I (not bcs it matter)
125, 
54136, 
12154, 
55681, 
1001796, 
26449, 
10185, 
32, 
1002526, 
6229, 
1001360, 
1001795, 
51293, 
28597, 
1001587, 
1002626, 
50379, 
52440, 
53492, 
21806, 
22447, 
591, 
51807, 
52419, 
55432, 
32083, 
51193, 
53927, 
55319, 
1004136, 
54896, 
16951, 
55050, 
30, 
1003489, 
16949, 
50620, 
31313, 
1003495, 
53965, 
54243, 
50338, 
15749, 
1004457, 
20184, 
50304, 
377, 
1000099, 
1164, 
54192, 
25860, 
1000011, 
60957, 
12116, 
1002655, 
15887, 
35056, 
2111, 
6811, 
1000112, 
1000109, 
6649, 
6989, 
19096, 
1003484, 
6492, 
32262, 
1004311, 
1003702, 
1000291, 
1002082, 
1004245, 
1002785, 
1004096, 
1003188, 
51053, 
6598, 
1002029, 
10556, 
54212, 
6831, 
55124, 
27761, 
27779, 
15958, 
53279, 
55330, 
16058, 
32567, 
# NORTH CALI 
16862, 
54121, 
12103, 
12426, 
1002532, 
53556, 
31169, 
12121, 
54294, 
1001600, 
27245, 
103, 
25156, 
52423, 
26772, 
27757, 
14203, 
55350, 
12999, 
14226, 
12710, 
24680, 
31423, 
13826, 
33897, 
12226, 
1000154, 
52417, 
33036, 
10403, 
13826, 
1004362, 
1002781, 
4111, 
14583, 
1003940, 
6992, 
6411, 
19383, 
345, 
13253, 
13561, 
13222, 
20431, 
10305, 
26147, 
55288, 
16785, 
6599, 
12671, 
55673, 
1002065, 
22379, 
11528, 
307, 
16641, 
22956, 
34517, 
54276, 
34787, 
1000673, 
7045, 
6898, 
1000748, 
1000119, 
50306, 
6836, 
30577, 
55514, 
54002, 
6833, 
34742, 
35093, 
19701, 
7121, 
6950, 
6480, 
1188, 
1004275, 
19956, 
14798, 
25393, 
28834, 
3669, 
1002312, 
1003892, 
1003711, 
33067, 
51113, 
34064, 
6782, 
1001181, 
23558, 
16056, 
52957, 
51615, 
6525, 
52618, 
1004143, 
19082, 
4307, 
55659, 
33492, 
34231, 
55173, 
32933, 
1004076, 
55112, 
50481, 
1001240, 
1000031, 
1002663, 
6685, 
1001467, 
1001458, 
1003797],
#### THEN , SKIPPED EVERYTHING ABOVE "SANFRANCISO BAY LABEL"
"Wyoming" : [
35140,
1004380,
55135
], 
"Texas" : [
35620, 
1003066, 
1002793, 
1000752, 
54743, 
18354, 
54937, 
54487, 
19963, 
34722, 
34631, 
1004406, 
33037, 
55754, 
52185, 
1003172, 
25029, 
54410, 
55311, 
51809, 
34084, 
1003937, 
34318, 
55635, 
52235, 
55536, 
32196, 
35594, 
10565, 
35771, 
13620, 
34691, 
35701, 
17739, 
10336, 
10344, 
34334, 
26735, 
52111, 
30303, 
1003157, 
50143, 
27214, 
52198, 
53101, 
50941, 
10317, 
1004493, 
53401, 
17914, 
50899, 
1001193, 
53923, 
1004458, 
52083, 
54539, 
50216, 
50456, 
32853, 
13694, 
18438, 
1198, 
12768, 
23017, 
1121, 
13283, 
12108, 
7038, 
50923, 
52383, 
55075, 
1001119, 
1003921, 
1001456, 
6409, 
1000515, 
1000523, 
6822, 
1002034, 
6938, 
4959, 
6988, 
1003680, 
1003583, 
1003164, 
1004077, 
13528, 
6692, 
64371002559, 
55741, 
1003066
],
"Illinois" : [
1002604, 
1001393, 
54251, 
55783, 
55712, 
19252, 
1004496, 
1001777, 
17226, 
25983, 
25988, 
1002366, 
25850, 
18425, 
25925, 
1004496, 
18741, 
12156, 
10545, 
54284, 
50223, 
19078, 
21905, 
55317, 
53206, 
12060, 
1003733, 
52634, 
10540, 
1002033, 
10194, 
14796, 
34220, 
19931, 
4085, 
16909, 
29004, 
1000105, 
2858, 
6280, 
6597, 
6987, 
32316, 
1002892, 
51388, 
50815, 
32471, 
12688, 
55655, 
52015, 
4019, 
11090, 
1004408, 
1004079, 
1004106, 
1004376, 
32555, 
53599, 
34287, 
1002093, 
54908, 
6629, 
6343
]}





In [59]:
import requests


def get_ip():
    response = requests.get('https://api64.ipify.org?format=json').json()
    return response["ip"]


def get_location():
    ip_address = "104.24.123.216"
    response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()
    location_data = {
        "ip": ip_address,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name")
    }
    return location_data


#print(get_location())

In [61]:
import pandas as pd 

df = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge.csv")

df_tmp = df[df["prb"] == 1004522]

print(df_tmp)

df2 = df[df["prb"].isin(dic["Wyoming"])]
print(df2)
print("CONTAINS " + str(df2.shape[0]) + " N.O. COLUMNS")


# "24998" "california" 
# "15887" "california"
# "6229" "california"

# "3649"  "Wyoming"


Empty DataFrame
Columns: [prb, Minlatency, MinIP, otherlatency1, otherIP1, otherlatency2, otherIP2, otherlatency3, otherIP3, otherlatency4, otherIP4, otherlatency5, otherIP5]
Index: []
Empty DataFrame
Columns: [prb, Minlatency, MinIP, otherlatency1, otherIP1, otherlatency2, otherIP2, otherlatency3, otherIP3, otherlatency4, otherIP4, otherlatency5, otherIP5]
Index: []
CONTAINS 0 N.O. COLUMNS
